In [2]:
from scipy.io.arff import loadarff
import pandas as pd
import numpy as np
import math

In [3]:
## Function Utils
def rand_centroids(K, X):
    # rand_centroids(K=Int, X=Float_array):
    # Return a numpy array of size K with each element 
    # being a normally random distributed var with mu and sigma calculated 
    # from the mean and std of the data X
    mean, std = np.mean(X, axis=0), np.std(X, axis=0)
    centroids = [np.random.normal(mean, std) for n in range(K)]
    return np.array(centroids)

def euc_distance(X, Y):    
    # euc_distance(X=Float_array, Y=Float_array):
    # Returns an array of euclidean distances, 
    # for the square root of the sum of the square of the differences
    # of array X and array Y
    diff = X - Y[:, np.newaxis]
    squared_diff = diff**2
    sum_squared_diff = squared_diff.sum(axis=2)
    return np.sqrt(sum_squared_diff)

def compute_centroids(K, C, X):
    # compute_centroids(K=Int, C=Float_array, X=Float_array)
    # Compute the centroids for cluster size K, centroid(s) C and data X
    # where a new centroid is calculated as the mean of the data points 
    # which share a common nearest centroid. Repeats until the sum of
    # the euc distances between centroids and points does not change, 
    # then returns the cluster of centroids
    D = euc_distance(X, C)
    CC = np.argmin(D, axis=0)
    C = np.array([new_centroid(k, X, CC) for k in range(K)])
    D2 = euc_distance(X, C)
    if (D.sum() == D2.sum()):
        return C, np.argmin(D2, axis=0)
    else:
        return compute_centroids(K, C, X)

def new_centroid(k, X, CC):
    # Returns a new centroid based on the mean of the points associated with it
    # if no points associated with it, generates a new one
    x = X[CC==k]
    if (len(x) > 0):
        return x.mean(axis=0)
    else: 
        return rand_centroids(1, X)[0]
    
def k_means(K, X):
    # k_means(K=Int, X=Float_array)
    # K-means for clust size K on dataset X using random initialised centroids
    # returns final cluster and predicted labels
    C = rand_centroids(K, X)
    return compute_centroids(K, C, X)

In [4]:
# Load iris data set values into a numpy array
iris_data, iris_meta = loadarff('./datasets/iris.arff')
data = np.array([[v[0], v[1], v[2], v[3]] for v in iris_data])
labels = np.unique([v[4] for v in iris_data])
labels_true = np.array([np.where(labels == v[4])[0][0] for v in iris_data])

In [5]:
# Perform k-means test
K = 2
clusters, labels_pred = k_means(K, data)
print(clusters)

[[ 6.30103093  2.88659794  4.95876289  1.69587629]
 [ 5.00566038  3.36037736  1.56226415  0.28867925]]


In [6]:
# SK-learn k-means comparison test
from sklearn.cluster import KMeans
sk_means = KMeans(n_clusters=K, init='random').fit(data)
print(sk_means.cluster_centers_)

[[ 5.00566038  3.36037736  1.56226415  0.28867925]
 [ 6.30103093  2.88659794  4.95876289  1.69587629]]


In [7]:
# Adjusted_rand_score
from sklearn.metrics import adjusted_rand_score
adjusted_rand_score(labels_true, labels_pred)

0.53992182942071232

In [8]:
# Calinski harabaz score
from sklearn.metrics import calinski_harabaz_score
calinski_harabaz_score(data, labels_pred)

513.30384335175665

In [9]:
# Perform scores for each metric and several clusters
MAX_K = 20
accuracies = []
for k in range(2, MAX_K):
    clusters, labels_pred = k_means(k, data)
    rand_score = adjusted_rand_score(labels_true, labels_pred)
    calinski = calinski_harabaz_score(data, labels_pred)
    accuracies.append([rand_score, calinski])
accuracies = np.array(accuracies)

In [10]:
# Plot the accuracies on a graph 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout

traces = [
    Scatter(x=range(2, MAX_K), y=accuracies[:,0], name = 'adjusted_rand_score'),
    Scatter(x=range(2, MAX_K), y=accuracies[:,1], name = 'calinski_score', yaxis='y2')
]

yaxis2=dict(side='right')
layout = Layout(
    title='K-means Metrics over cluster size 2-20',
    xaxis=dict(title='cluster_size'),
    yaxis=dict(title='adjusted_rand_score'),
    yaxis2=dict(title='calinski_score', overlaying='y', side='right')
)

fig = Figure(data=traces, layout=layout)
plot(fig)

'file:///Users/jnalexander/Projects/MAI-machine-learning/temp-plot.html'

In [13]:
''' 
Details about the implementation of your algorithms, including the decisions made during the
implementation and the setup of the different parameters

K-means

The implementation was mainly built from following the course notes on k-means, 
as well as Coursera's tutorial:
https://www.coursera.org/learn/machine-learning/lecture/93VPG/k-means-algorithm

The algorithm was divided into 5 key functions:
    k_means(K, X)
        - for initializing a new clustering, K is the number of clusters wanted, X is the dataset
        to cluster.
        - the function simply initiates the centroids randomly and then computes the clusters.
        - the function returns both the clusters and an array of the predicted cluster assignment
        for each datapoint.

    rand_centroids(K, X): 
        - this function returns K generated centroids from data X.
        - it does this by calculating the standard deviation and mean along each dimension of X
        and then creates a centroid by choosing a normally distributed random value with the 
        standard deviation and mean of the dimension for the centroid.
        - a normal distribution was chosen as this would provide a better initial guess, where
        better means that the centroid would be closer and converge faster to the final clusters.
    
    euc_distance(X, Y):   
        - this function returns the square root of the sum of the squares of the differences in 
        values of X and Y, across the same dimensions. It was used as it is a distance metric for 
        computing the distances from points and clusters in the k-means algorithm.

    compute_centroids(K, C, X):
        - given the cluster size, initial centroids and data, this function computes the centroids
        and the array of the predicted centroid assignment for each datapoint.
        - it uses a euclidean distance metric to calculate the distance between centroids and
        points, then it assigns each point to its nearest centroid, before calling itself again,
        until a cluster array is produced that has the same euclidean distance sum as the one
        before it. 
    
    new_centroid(k, X, CC):
        - this function creates a new centroid according to the cluster index, the data points
        and the map of closest centroids to points. 
        - it does this by selecting the points associated with a centroid, calculating their mean,
        along each dimension, and returning this as an array.
        - important to note is that if a centroid is chosen that has no points associated with it,
        it was chosen to create a new centroid, based on the mean and var of the data 
        (see rand_centroids function). This was done to ensure that the k-means algorithm always
        returned the same cluster size as was called for.
'''

" \nDetails about the implementation of your algorithms, including the decisions made during the\nimplementation and the setup of the different parameters\n\nK-means\n\nThe implementation was mainly built from following the course notes on k-means, \nas well as Coursera's tutorial:\nhttps://www.coursera.org/learn/machine-learning/lecture/93VPG/k-means-algorithm\n\nThe algorithm was divided into 5 key functions:\n    k_means(K, X)\n        - for initializing a new clustering, K is the number of clusters wanted, X is the dataset\n        to cluster.\n        - the function simply initiates the centroids randomly and then computes the clusters.\n        - the function returns both the clusters and an array of the predicted cluster assignment\n        for each datapoint.\n\n    rand_centroids(K, X): \n        - this function returns K generated centroids from data X.\n        - it does this by calculating the standard deviation and mean along each dimension of X\n        and then creates a